In [1]:
import pandas as pd 
import tensorflow as tf
from PIL import Image
import numpy as np
# Importing the test dataset
y_test = pd.read_csv(r"D:\Test.csv")

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

y_test = labels
data=[]

# Retreiving the images
with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open(img)
        image = image.resize([64,64])
        data.append(np.array(image))

X_test=np.array(data)



In [2]:
import numpy as np
from keras.models import load_model
import pickle
from skimage.feature import hog
from skimage import color
from sklearn.metrics import accuracy_score
import cv2

# Nạp các mô hình
model_CNN = load_model(r"C:\Users\Admin\Downloads\traffic_classifier333.h5")
with open(r"C:\Users\Admin\Downloads\SVM.pkl", 'rb') as f:
    model_SVM = pickle.load(f)
with open(r"C:\Users\Admin\Downloads\RF.pkl", 'rb') as f:
    model_RF = pickle.load(f)

# Hàm trích xuất đặc trưng HOG
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img_gray = color.rgb2gray(cv2.resize(img, (64, 64)))  # Đảm bảo ảnh kích thước (64,64)
        features = hog(img_gray, block_norm='L2-Hys', pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(features)
    return np.array(hog_features)

# Hàm dự đoán từ các mô hình
def get_predictions(X):
    # Dự đoán với mô hình CNN (reshape về 4D nếu cần)
    cnn_preds = np.argmax(model_CNN.predict(X.reshape(-1, 64, 64, 3)), axis=1)
    
    # Trích xuất đặc trưng HOG cho SVM và RF
    X_hog = extract_hog_features(X)  # Xử lý để có đúng đặc trưng HOG kích thước 1764
    svm_preds = model_SVM.predict(X_hog)
    rf_preds = model_RF.predict(X_hog)
    
    return cnn_preds, svm_preds, rf_preds

# Hàm tính toán voting thủ công
def manual_voting(cnn_preds, svm_preds, rf_preds):
    final_predictions = []
    for i in range(len(cnn_preds)):
        # Đếm số phiếu của từng nhãn từ các mô hình
        votes = [cnn_preds[i], svm_preds[i], rf_preds[i]]
        # Chọn nhãn có số phiếu nhiều nhất
        final_predictions.append(np.bincount(votes).argmax())
    return np.array(final_predictions)




In [3]:
# Thực hiện dự đoán trên tập con
cnn_preds, svm_preds, rf_preds = get_predictions(X_test)
predictions = manual_voting(cnn_preds, svm_preds, rf_preds)

# Tính độ chính xác
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy on test set: {accuracy}")

375/375 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step
Accuracy on test set: 0.9929107589658048
